In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import cv2
import matplotlib.patches as patches
import math
import heapq
from matplotlib import animation
import os
import time

In [ ]:
def in_circle(x, y, cx, cy, r):
    # Check if (x, y) is in the circle centered at (cx, cy) with radius r
    return (x - cx)**2 + (y - cy)**2 <= r**2


def in_partial_ring(x, y,
                    cx, cy,
                    outer_r, inner_r,
                    start_deg, end_deg):
    """
    Check if (x, y) is in the partial ring defined by
           - circle centered at (cx, cy) with 
               - outer_radius  = outer_r
               - inner_r       = inner_r
               - angles between start_deg and end_deg
     Coords wrt bottom-left origin
    """

    dx, dy    = x - cx, y - cy          # Vector from center of both outer/inner circles to point (x, y)
    angle_rad = math.atan2(dy, dx)      # Angle in radians from circle's center to point (x, y)
    angle_deg = math.degrees(angle_rad) # Convert angle to degrees

    pt_in_outer_circle = in_circle(x, y, cx, cy, outer_r) # Check if point is in outer circle
    pt_in_inner_circle = in_circle(x, y, cx, cy, inner_r) # Check if point is in inner circle

    if angle_deg < 0: # Convert negative angles to positive
        angle_deg += 360

    pt_is_in_ring      = pt_in_outer_circle and not pt_in_inner_circle  # Check if point is in ring
    pt_is_in_deg_range = start_deg <= angle_deg <= end_deg              # Check if point is in degree range

    # If points is in ring and in degree range, return True, else return False
    if pt_is_in_deg_range:
        if pt_is_in_ring:
            return True
    return False


def in_right_half_circle(x, y, cx, cy, radius):
    # Check if (x, y) is in the right half of the circle centered at (cx, cy) with radius r
    # Coords wrt bottom-left origin
    inside_circle = (x - cx)**2 + (y - cy)**2 <= radius**2
    inside_right_half   = x >= cx
    return inside_circle and inside_right_half


def to_the_right(ex1, ey1, ex2, ey2, x, y):
    # Helper function to in_parallellogram, checks if point (x, y) is to the left of the line defined by points (px1, py1) and (px2, py2)
    # (ex1, ey1) and (ex2, ey2) define the input edge of parallelogram
    # Vector1: (edge_x, edge_y) = (ex2 - ex1, ey2 - ey1)   points in direction of input edge from start to end
    # Vector2: (pt_x, pt_y) = (x - ex1, y - ey1)       points is direction from edge start to point (x, y)
    # Cross Product of 2-D vector defined by 
    #               (edge_x, edge_y, 0) x (pt_x, pt_y, 0) = (0, 0, edge_x * pt_y - edge_y * pt_x)
    # Coords wrt bottom-left origin
    
    edge_x, edge_y = ex2 - ex1, ey2 - ey1
    pt_x, pt_y     = x - ex1, y - ey1
    cross          = (edge_x * pt_y) - (edge_y * pt_x)

    # If cross product is positive, point is to the right of the edge
    # If cross product is negative, point is to the left of the edge
    return cross <= 0


def in_parallelogram(x, y, A, B, C, D):
    # Check if (x, y) is inside the parallelogram defined by the four points A, B, C, D
    # A, B, C, D are defined in clockwise order, so we check if (x, y) is to the right of each edge
    # Coords wrt bottom-left origin

    return (to_the_right(A[0], A[1], B[0], B[1], x, y) and
            to_the_right(B[0], B[1], C[0], C[1], x, y) and
            to_the_right(C[0], C[1], D[0], D[1], x, y) and
            to_the_right(D[0], D[1], A[0], A[1], x, y))


def in_rectangle(x, y, xmin, xmax, ymin, ymax):
    # Returns True if (x, y) is inside rectangle defined by (xmin, ymin), (xmax, ymax) corners
    # Coords wrt bottom-left origin
    return (x >= xmin) and (x <= xmax) and (y >= ymin) and (y <= ymax)


def in_E(x, y, start_x, start_y):
    # Check if x, y is inside the letter 'E'
    # Coords wrt bottom-left origin
    R_v   = in_rectangle(x, y, start_x, start_x+5,  start_y,    start_y+25)    # vertical bar
    R_top = in_rectangle(x, y, start_x, start_x+13, start_y+20, start_y+25) # top horizontal
    R_mid = in_rectangle(x, y, start_x, start_x+13, start_y+10, start_y+15)  # middle horizontal
    R_bot = in_rectangle(x, y, start_x, start_x+13, start_y,    start_y+ 5)   # bottom horizontal
    
    return R_v or R_top or R_mid or R_bot


def in_1(x, y, start_x, start_y):
    # Check if x, y is inside our coordinate for the number 1 in map_data
    # Coords wrt bottom-left origin
    R   = in_rectangle(x, y, start_x, start_x+5,  start_y,    start_y+28)    # vertical bar

    return R


def in_N(x, y, start_x, start_y):
    # Check whether (x, y) is inside the letter 'N'
    # Coords wrt bottom-left origin
    # Define N's Diagonal as parallelograms of points defined in clockwise order
    A = (start_x,    start_y+25)
    B = (start_x+5,  start_y+25)
    C = (start_x+20, start_y)
    D = (start_x+15, start_y)
    
    # Check if (x, y) is in our geometric definition of N
    R_left   = in_rectangle(x, y,    start_x, start_x+5,  start_y,   start_y+25)    # Left  Vertical Bar of N
    R_right  = in_rectangle(x, y, start_x+15, start_x+20,  start_y,  start_y+25)    # Right Vertical Bar of N
    diagonal = in_parallelogram(x, y, A, B, C, D)                                   # Diagonal of N
    return R_left or R_right or diagonal


def in_P(x, y, start_x, start_y):
    # Check whether (x, y) is inside the letter 'P'
    # Coords wrt bottom-left origin

    radius = 6                     # Radius of our P's Half-Circle
    cx     = start_x + 5           # Half-Circle Center X Coordinate (On top of our P's Vertical Bar)
    cy     = start_y + 25-radius   # Half-Circle Center Y Coordinate (On top of our P's Vertical Bar)

    #Check if points is in our geometrical definition of P
    bar = in_rectangle(x, y,       # Vertical bar of our P
                       xmin=start_x,
                       xmax=start_x+5,
                       ymin=start_y,
                       ymax=start_y+25)

    top_half_circle = in_right_half_circle(x, y, cx, cy, radius) # Half-Circle of P

    return bar or top_half_circle


def in_M(x, y, start_x, start_y):
    # Check whether (x, y) is inside the letter 'M'
    # Coords wrt bottom-left origin
    # Diagonals of M Defined as parallelograms of points defined clockwise

    # Values below were defined in project prompt or were manually checked to get shape close to project prompt
    m_gap         = 5           # Horizontal Gap between the two vertical bars of M and the box connecting the diagonals in the middle
    bottom_w      = 7           # Width of the bottom rectangle in the middle of the M connecting the two diagonals 
    bottom_offset = 5 + m_gap   # Offset from the start_x to the start of the bottom rectangle in the middle
    bottom_w      = 7           # Width of the bottom rectangle in the middle of the M connecting the two diagonals

    second_box_offset = bottom_offset + bottom_w + m_gap # Leftmost X coord of second vertical bar of M

    # First Vertical Box to Middle Rectangle Box Diagonal:
    A = (start_x,                start_y+25) # Diagonal 1, Top Left
    B = (start_x+5,              start_y+25) # Diagonal 1, Top Right
    C = (start_x+bottom_offset+1, start_y+5) # Diagonal 1, Bottom Right
    D = (start_x+bottom_offset,   start_y+0) # Diagonal 1, Bottom Left

    # Middle Rectangle Box Diagonal to Second Vertical Box:
    A1 = (start_x+second_box_offset,       start_y+25) # Diagonal 2, Top Left
    B1 = (start_x+second_box_offset+5,     start_y+25) # Diagonal 2, Top Right
    C1 = (start_x+bottom_offset+bottom_w,     start_y) # Diagonal 2, Bottom Left
    D1 = (start_x+bottom_offset+bottom_w-1, start_y+5) # Diagonal 2, Bottom Right
   
    # Check if (x, y) is in our geometric definition of M:
    R_left   = in_rectangle(x, y,    
                    start_x, start_x+5,  start_y,   start_y+25)     # First Vertical Bar
    
    R_right  = in_rectangle(x, y,                                   # Second Vertical Bar
                            start_x+second_box_offset, 
                            start_x+second_box_offset+5,  
                            start_y,  
                            start_y+25)    
    
    R_bottom = in_rectangle(x, y,                                   # Middle Retangle between two vertical bars
                            start_x+bottom_offset, 
                            start_x+bottom_offset+bottom_w,
                            start_y, 
                            start_y+5) 

    diagonal1 = in_parallelogram(x, y, A,  B,  C,  D)   # From First Vertical Bar to Middle Retangle between the two vertical bars
    diagonal2 = in_parallelogram(x, y, A1, B1, C1, D1)  # Middle Rectangle Box Diagonal to Second Vertical Box
    return R_left or R_right or R_bottom or diagonal1 or diagonal2
    

def in_6(x, y, start_x, start_y):
    # Check whether (x, y) is inside the number '6'
    # Coords wrt bottom-left origin

    cx = start_x + 20   # Adjusted Manually to get shape close to project prompt
    cy = start_y + 10   # Adjusted Manually to get shape close to project prompt

    outer_r = 21.5      # From Project Prompt
    inner_r = 16.5      # From Project Prompt
    start_deg, end_deg = 120, 180    # Degrees to sweep for curly top part of 6 curves, Adjusted Manually to get shape close to project prompt
    bottom_cx  = start_x + 7         # Adjusted Manually to get shape close to project prompt
    bottom_cy  = start_y + 7         # Adjusted Manually to get shape close to project prompt
    bottom_r   = 9                   # From Project Prompt

    # Define Circle centered at outer tip of 6 and inner tip of 6 
    tip_radius = 2.5    # From Project Prompt
    outer_tip  = cx + outer_r * np.cos(np.deg2rad(start_deg)), cy + outer_r * np.sin(np.deg2rad(start_deg))
    inner_tip  = cx + inner_r * np.cos(np.deg2rad(start_deg)), cy + inner_r * np.sin(np.deg2rad(start_deg))
    center_tip = (outer_tip[0] + inner_tip[0]) / 2, (outer_tip[1] + inner_tip[1]) / 2
    
    # Checks if (x, y) is in curled top part of 6
    curly_top     =  in_partial_ring(x, y, cx, cy, outer_r, inner_r,
                                     start_deg, end_deg)
    
    # Checks if (x, y) is in circular part of the bottom of the 6
    bottom_circle = in_circle(x, y, bottom_cx, bottom_cy, bottom_r)

    # Checks if (x, y) is in circular part at end of upper curled part of 6
    tip_circle    = in_circle(x, y, center_tip[0], center_tip[1], tip_radius)

    return curly_top, bottom_circle, tip_circle


def draw(map_img, start_x, start_y, letter='E'):
    h, w = map_img.shape
    for py in range(h):
        for px in range(w):
            x_bl = px
            y_bl = py

            if letter == 'E': # Draw E
                if in_E(x_bl, y_bl, start_x, start_y):
                    map_img[py, px] = 0

            elif letter == 'N': # Draw N
                if in_N(x_bl, y_bl, start_x, start_y):
                    map_img[py, px] = 0

            elif letter == 'P': # Draw P
                if in_P(x_bl, y_bl, start_x, start_y):
                    map_img[py, px] = 0

            elif letter == 'M': # Draw M
                if in_M(x_bl, y_bl, start_x, start_y):
                    map_img[py, px] = 0

            elif letter == '6': # Draw 6
                curly_top, bottom_circle, tip_circle = in_6(x_bl, y_bl, start_x, start_y)
                if curly_top or bottom_circle or tip_circle:
                    map_img[py, px] = 0
            
            elif letter == '1': # Draw 1
                if in_1(x_bl, y_bl, start_x, start_y):
                    map_img[py, px] = 0
            
    return map_img


def add_buffer(map_img, buffer_size=5):
    # Add 2 pixels to our map_data by dilating obstacles with a circular kernel with radius=buffer_size
    map_img_copy = map_img.copy()

    # Create Circular Dilation Kernel, for morphology operations, we need a single center pixel, and a 2x2 circle has no center pixel, so we use a 3x3 circle 
    # The center pixel is 1 pixel, and the 8 surrounding pixels extend 1 pixel, so total radius is 2
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (buffer_size*2+1, buffer_size*2+1))

    # In OpenCV, white(255) is treated as the foreground, Black(0) is Obstacle Space, so we need to invert the colors
    map_with_clearance = cv2.dilate(255 - map_img_copy, kernel)

    # Invert back (to original representation: obstacles=0, free=255)
    map_img_copy = 255 - map_with_clearance
    return map_img_copy

